# Step 4: Classify Contracts

This notebook uses the assembled context from Step 3 to classify each contract document.

**What the LLM determines for each document:**
1. Is this a master agreement?
2. Does it have amendments?
3. What is the initial expiry date?
4. What is the final expiry date (after all amendments)?
5. What are the applicable amendments and how do they affect the expiry?

The LLM gets the document's own info, preamble, folder-level related documents, and semantically similar documents. It weighs folder documents more heavily since amendments are almost always in the same folder as their master agreement.

**Before you run this:**
- Steps 1-3 must be complete
- The `assembled` table must exist with data

**Output table:**
- `classified` -- master agreement flag, amendment list, initial/final expiry dates, rationale, confidence score

## Configuration

In [ ]:
dbutils.widgets.text("catalog", "shm", "Catalog")
dbutils.widgets.text("schema", "contract", "Schema")
dbutils.widgets.text("batch_size", "100", "Batch Size")
dbutils.widgets.text("max_input_char", "400000", "Max Input Characters")

catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
batch_size = dbutils.widgets.get("batch_size")
max_input_char = dbutils.widgets.get("max_input_char")

---
## Run Classification

The LLM receives all the assembled context and returns a structured classification. It skips documents already in the `classified` table, so you can re-run safely.

Confidence scale:
- **5** = perfectly confident, clear folder documentation
- **4** = confident based on folder documents
- **3** = some uncertainty, may rely on semantic search
- **2** = quite uncertain, doubts about accuracy
- **1** = very little information

In [ ]:
CREATE TABLE IF NOT EXISTS IDENTIFIER(:catalog || '.' || :schema || '.classified') (
  path STRING,
  is_master_agreement BOOLEAN,
  has_amendments BOOLEAN,
  initial_master_agreement_expiry_date STRING,
  final_expiry_date STRING,
  final_expiry_date_rationale STRING,
  final_expiry_date_source_path STRING,
  amendments ARRAY<STRUCT<
    amendment_path: STRING,
    changes_master_agreement_expiry: BOOLEAN,
    new_master_agreement_expiry: STRING,
    agreement_document_type: STRING
  >>,
  rationale STRING,
  confidence INT
);
 
MERGE INTO IDENTIFIER(:catalog || '.' || :schema || '.classified') AS target
USING (
  SELECT
    path,
    classification.is_master_agreement,
    classification.has_amendments,
    classification.initial_master_agreement_expiry_date,
    classification.final_expiry_date,
    classification.final_expiry_date_rationale,
    classification.final_expiry_date_source_path,
    classification.amendments,
    classification.rationale,
    classification.confidence
  FROM (
    SELECT
      path,
      from_json(AI_QUERY(
        "databricks-claude-sonnet-4-5",
        SUBSTRING(CONCAT(
        'You are a contractual document classification expert. You will be provided with a document, key information, the preamble (first 100 words), and two sets of related documents. The first is documents within the vendor folder with the key information and preamble from those documents. The second is key information of semantically similar documents.
        
        Use this information to answer the following questions:
        1. Is this document a master agreement?
        2. Are there any amendments to this master agreement?
        3. What is the initial expiry date of this agreement?
        4. What is the final expiry date of this agreement given all of the amendments?
        5. What are the applicable amendments?
        
        List all the amendments, and if this amendment changes the final expiry date. Amendments may have other changes than the date that do not affect the master agreement, for example rate changes. Ignore non-agreement documents like confidentiality (CA), non-disclosure (NDA), scope of work (SOW), or termination documents when determining amendments.
        
        # FINAL DATE
        The most critical part of your task is to identify the final date of the agreement based on the master agreement and amendments. Think carefully about this given the provided information. It includes information of documents from within the same vendor folder (VENDOR DOC), and semantically similar documents (OTHER DOC). The other documents are semantically similar documents that may or may not be related to the contract. Weigh the vendor docs more heavily and do your best to provide the final expiry date, which should also be after the initial expiry date. Provide the path associated with the document info that you reference for the final date.
 
        # ADDITIONAL INSTRUCTIONS
        Note that there are numerous files within the same folder that are more likely to be related to the contract than others. The folder documents should weigh much more than the other semantically related documents.  
 
        For amendments, determine their sequence (e.g., Amendment 1, Amendment 2) using the date found inside the document key information to establish chronological order. These amendments should have a path.
 
        Use document content for classification, not file names.
        Ignore duplicates.
        
        Provide your rationale and confidence on a scale of 1 to 5.
 
        Think carefully and review the work.

        Provide all dates in YYYYMMDD format. Leave blank if dates are unknown.
 
        Provide your response in json format with the following fields:
        {
          "is_master_agreement": boolean,
          "has_amendments": boolean,
          "initial_master_agreement_expiry_date": date,
          "final_expiry_date": date,
          "final_expiry_date_rationale": string,
          "final_expiry_date_source_path": string,
          "amendments": [
            {
              "amendment_path": string,
              "changes_master_agreement_expiry": boolean,
              "new_master_agreement_expiry": date,
              "agreement_document_type": string
            },
            ...
          ],
          "rationale": string,
          "confidence": integer
        }
        ',
        '# PATH: ', path,
        '\n \n# KEY INFO:\n', doc_info,
        '\n \n# PREAMBLE:\n ', preamble,
        '\n \n# NUMBER OF FOLDER DOCS:\n', CAST(other_folder_docs AS STRING),
        '\n \n# FOLDER DOC KEY INFO:\n', other_doc_infos,
        '\n \n# FOLDER DOC PREAMBLE:\n ', other_preambles,
        '\n \n# OTHER DOC KEY INFO ', vs_results,
        '\n \n# OTHER DOC PREAMBLE ', vs_preamble_results
        ),1,:max_input_char),
        responseFormat => 'STRUCT<results:STRUCT<
          is_master_agreement:BOOLEAN,
          has_amendments:BOOLEAN,
          initial_master_agreement_expiry_date:STRING,
          final_expiry_date: STRING,
          final_expiry_date_rationale: string,
          final_expiry_date_source_path: string,
          amendments:ARRAY<STRUCT<
            amendment_path:STRING,
            changes_master_agreement_expiry:BOOLEAN,
            new_master_agreement_expiry:STRING,
            agreement_document_type:STRING
          >>,
          rationale:STRING,
          confidence:INT
        >>'
      ), 'STRUCT<
          is_master_agreement:BOOLEAN,
          has_amendments:BOOLEAN,
          initial_master_agreement_expiry_date:STRING,
          final_expiry_date: STRING,
          final_expiry_date_rationale: string,
          final_expiry_date_source_path: string,
          amendments:ARRAY<STRUCT<
            amendment_path:STRING,
            changes_master_agreement_expiry:BOOLEAN,
            new_master_agreement_expiry:STRING,
            agreement_document_type:STRING
          >>,
          rationale:STRING,
          confidence:INT
        >'
      ) AS classification
    FROM IDENTIFIER(:catalog || '.' || :schema || '.assembled') a
    LEFT ANTI JOIN IDENTIFIER(:catalog || '.' || :schema || '.classified') c
      ON a.path = c.path
    LIMIT CAST(:batch_size AS INT)
  )
) AS source
ON target.path = source.path
WHEN NOT MATCHED THEN INSERT *

In [ ]:
-- Check results
SELECT 
  COUNT(*) as total,
  SUM(CASE WHEN is_master_agreement THEN 1 ELSE 0 END) as master_agreements,
  AVG(confidence) as avg_confidence
FROM IDENTIFIER(:catalog || '.' || :schema || '.classified')